In [1]:
import pandas as pd
import json
from collections import defaultdict

## 1. Start and end stations for which the predictions are better


In [136]:
df = pd.read_csv('good_startstations.csv')

In [137]:
df2 = pd.read_csv('citibike_stations.csv')

In [138]:
df = df[['startstationid','startstationname','startstationlatitude', 'startstationlongitude', 
'endstationname', 'endstationid']]
df.head()

,startstationid,startstationname,startstationlatitude,startstationlongitude,endstationname,endstationid
0,3255,8 Ave & W 31 St,40.750585,-73.994685,E 25 St & 2 Ave,507
1,402,Broadway & E 22 St,40.740343,-73.989551,E 20 St & FDR Drive,487
2,3255,8 Ave & W 31 St,40.750585,-73.994685,9 Ave & W 18 St,434
3,263,Elizabeth St & Hester St,40.717290,-73.996375,E 9 St & Avenue C,394
4,402,Broadway & E 22 St,40.740343,-73.989551,W 22 St & 10 Ave,462


In [139]:
df2.head()

,Unnamed: 0,startstationlatitude,startstationlongitude,startstationname,startstationid
0,0,40.750664,-74.001768,10 Ave & W 28 St,489
1,1,40.668627,-73.987001,10 St & 5 Ave,3317
2,2,40.666208,-73.981999,10 St & 7 Ave,3306
3,3,40.751396,-74.005226,11 Ave & W 27 St,458
4,4,40.760301,-73.998842,11 Ave & W 41 St,478


In [140]:
unique_starts = list(df.startstationid.unique())
unique_ends = list(df.endstationid.unique())

In [142]:
coordinates_start = df2[df2.startstationid.isin(unique_starts)].drop('Unnamed: 0', 1)

In [143]:
coordinates_stop = df2[df2.startstationid.isin(unique_ends)].drop('Unnamed: 0', 1)

In [144]:
coordinates_start.to_csv('coordinates_start_wd_f.csv')
coordinates_stop.to_csv('coordinates_stop_wd_f.csv')

## 2. The .json for visualisation of probabilities


* match probas.csv to the df containing station names and their coordinates
* convert them into a json with structure:  
{startstation:{latitude:coordinate, longitude: coordinate, endstations:{station:probability}  
Only thehe ten end stations with the highest probability will be included.

In [146]:
probas = pd.read_csv('probas.csv')

In [147]:
probas.head()

,startlat,startlong,1 Ave & E 15 St,1 Ave & E 18 St,1 Ave & E 30 St,1 Ave & E 68 St,11 Ave & W 27 St,12 Ave & W 40 St,5 Ave & E 29 St,6 Ave & W 33 St,...,W 22 St & 8 Ave,W 25 St & 6 Ave,W 26 St & 10 Ave,W 26 St & 8 Ave,W 38 St & 8 Ave,W 4 St & 7 Ave S,W Broadway & Spring St,Washington St & Gansevoort St,West St & Chambers St,West Thames St
0,40.704633,-74.013617,0.002934,0.010118,0.000164,0.003739,0.010347,0.074872,0.000204,0.000055,...,0.006014,0.002138,0.002821,0.003347,0.008992,0.002201,0.007638,0.012341,0.029966,0.065438
1,40.705693,-74.016777,0.000192,0.001172,0.004149,0.000109,0.013253,0.030247,0.004013,0.000000,...,0.010077,0.000550,0.017170,0.002660,0.005852,0.007374,0.006410,0.015977,0.054394,0.042981
2,40.708347,-74.017134,0.000078,0.000229,0.000327,0.000100,0.013554,0.022046,0.005386,0.000034,...,0.004729,0.002579,0.017093,0.002882,0.004494,0.011836,0.004317,0.017796,0.046115,0.038778
3,40.711512,-74.015756,0.004755,0.001387,0.000117,0.000052,0.006762,0.033894,0.002283,0.000028,...,0.002629,0.007212,0.004285,0.005584,0.001561,0.028263,0.011890,0.042987,0.028105,0.106470
4,40.714275,-73.989900,0.059302,0.000966,0.014966,0.000985,0.002975,0.000393,0.009675,0.000514,...,0.000445,0.002383,0.003594,0.000632,0.000535,0.003691,0.008968,0.002032,0.001436,0.004985


In [148]:
probas.shape

(92, 94)

In [150]:
df3 = df2.rename(columns={'startstationlatitude':'startlat', 'startstationlongitude': 'startlong'}).drop('Unnamed: 0', 1)

In [151]:
df3.head()

,startlat,startlong,startstationname,startstationid
0,40.750664,-74.001768,10 Ave & W 28 St,489
1,40.668627,-73.987001,10 St & 5 Ave,3317
2,40.666208,-73.981999,10 St & 7 Ave,3306
3,40.751396,-74.005226,11 Ave & W 27 St,458
4,40.760301,-73.998842,11 Ave & W 41 St,478


In [153]:
df4 = pd.merge(probas, df3.drop('startstationid', 1), on=['startlat', 'startlong'], how='left')

In [156]:
df4.head()

,startlat,startlong,1 Ave & E 15 St,1 Ave & E 18 St,1 Ave & E 30 St,1 Ave & E 68 St,11 Ave & W 27 St,12 Ave & W 40 St,5 Ave & E 29 St,6 Ave & W 33 St,...,W 25 St & 6 Ave,W 26 St & 10 Ave,W 26 St & 8 Ave,W 38 St & 8 Ave,W 4 St & 7 Ave S,W Broadway & Spring St,Washington St & Gansevoort St,West St & Chambers St,West Thames St,startstationname
0,40.704633,-74.013617,0.002934,0.010118,0.000164,0.003739,0.010347,0.074872,0.000204,0.000055,...,0.002138,0.002821,0.003347,0.008992,0.002201,0.007638,0.012341,0.029966,0.065438,Broadway & Battery Pl
1,40.705693,-74.016777,0.000192,0.001172,0.004149,0.000109,0.013253,0.030247,0.004013,0.000000,...,0.000550,0.017170,0.002660,0.005852,0.007374,0.006410,0.015977,0.054394,0.042981,Little West St & 1 Pl
2,40.708347,-74.017134,0.000078,0.000229,0.000327,0.000100,0.013554,0.022046,0.005386,0.000034,...,0.002579,0.017093,0.002882,0.004494,0.011836,0.004317,0.017796,0.046115,0.038778,West Thames St
3,40.711512,-74.015756,0.004755,0.001387,0.000117,0.000052,0.006762,0.033894,0.002283,0.000028,...,0.007212,0.004285,0.005584,0.001561,0.028263,0.011890,0.042987,0.028105,0.106470,South End Ave & Liberty St
4,40.714275,-73.989900,0.059302,0.000966,0.014966,0.000985,0.002975,0.000393,0.009675,0.000514,...,0.002383,0.003594,0.000632,0.000535,0.003691,0.008968,0.002032,0.001436,0.004985,Canal St & Rutgers St


In [158]:
j = df4.set_index('startstationname')
j.head()

,startlat,startlong,1 Ave & E 15 St,1 Ave & E 18 St,1 Ave & E 30 St,1 Ave & E 68 St,11 Ave & W 27 St,12 Ave & W 40 St,5 Ave & E 29 St,6 Ave & W 33 St,...,W 22 St & 8 Ave,W 25 St & 6 Ave,W 26 St & 10 Ave,W 26 St & 8 Ave,W 38 St & 8 Ave,W 4 St & 7 Ave S,W Broadway & Spring St,Washington St & Gansevoort St,West St & Chambers St,West Thames St
startstationname,,,,,,,,,,,,,,,,,,,,,
Broadway & Battery Pl,40.704633,-74.013617,0.002934,0.010118,0.000164,0.003739,0.010347,0.074872,0.000204,0.000055,...,0.006014,0.002138,0.002821,0.003347,0.008992,0.002201,0.007638,0.012341,0.029966,0.065438
Little West St & 1 Pl,40.705693,-74.016777,0.000192,0.001172,0.004149,0.000109,0.013253,0.030247,0.004013,0.000000,...,0.010077,0.000550,0.017170,0.002660,0.005852,0.007374,0.006410,0.015977,0.054394,0.042981
West Thames St,40.708347,-74.017134,0.000078,0.000229,0.000327,0.000100,0.013554,0.022046,0.005386,0.000034,...,0.004729,0.002579,0.017093,0.002882,0.004494,0.011836,0.004317,0.017796,0.046115,0.038778
South End Ave & Liberty St,40.711512,-74.015756,0.004755,0.001387,0.000117,0.000052,0.006762,0.033894,0.002283,0.000028,...,0.002629,0.007212,0.004285,0.005584,0.001561,0.028263,0.011890,0.042987,0.028105,0.106470
Canal St & Rutgers St,40.714275,-73.989900,0.059302,0.000966,0.014966,0.000985,0.002975,0.000393,0.009675,0.000514,...,0.000445,0.002383,0.003594,0.000632,0.000535,0.003691,0.008968,0.002032,0.001436,0.004985


In [159]:
from collections import defaultdict

results = defaultdict(lambda: defaultdict(dict))
for index in j.index.values:
    results[index] = {'endstations': defaultdict(dict)}
    columns = j.loc[index]
    for coordinates in columns.index.values[:2]:
        results[index][coordinates] = columns.loc[coordinates]
    for endstation in columns.index.values[3:]:
        probability = columns.loc[endstation]
        results[index]['endstations'][endstation] = probability

In [163]:
for k, v in results.items():
    prob_stat = sorted([(key, value) for key, value in v['endstations'].items()], key= lambda x: x[1], reverse=True)
    highest_prob = prob_stat[:10]
    results[k]['endstations'] = {key:{'probability': value} for key, value in highest_prob}

In [178]:
df_indexed = df3.set_index(df3.startstationname)

In [181]:
df_indexed.head()

,startlat,startlong,startstationname,startstationid
startstationname,,,,
10 Ave & W 28 St,40.750664,-74.001768,10 Ave & W 28 St,489
10 St & 5 Ave,40.668627,-73.987001,10 St & 5 Ave,3317
10 St & 7 Ave,40.666208,-73.981999,10 St & 7 Ave,3306
11 Ave & W 27 St,40.751396,-74.005226,11 Ave & W 27 St,458
11 Ave & W 41 St,40.760301,-73.998842,11 Ave & W 41 St,478


In [183]:
# add the endstationscoordinates
for start, info in results.items():
    for end, prob in info['endstations'].items():
        lat = df_indexed.loc[end].startlat
        lon = df_indexed.loc[end].startlong
        results[start]['endstations'][end]['latitude'] = lat
        results[start]['endstations'][end]['longitude'] = lon
    

In [185]:
with open('relations.json', 'w') as fp:
    json.dump(results, fp)